In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/student-marks-dataset/Student.csv")
print(df.head())
print(f"shape of the dataset is : {df.shape}")

In [ ]:
df.isnull().sum()

In [ ]:
df.Result.value_counts()

In [ ]:
#imbalanced data so we need to balance it

Result_0 = df[df["Result"] == 0]
Result_1 = df[df["Result"] == 1]
from sklearn.utils import resample
df_minority_upsampled = resample(Result_1, replace = True, n_samples = 755)
new_df = pd.concat([Result_0, df_minority_upsampled])

from sklearn.utils import shuffle
new_df = shuffle(new_df)

In [ ]:
new_df.Result.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense


In [ ]:
X = new_df.drop(['Result'], axis = 1)
y = new_df['Result']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from tensorflow import keras

model = keras.Sequential(
    [ keras.layers.Dense(
            10, activation="relu", input_shape=[3]
        ),
        keras.layers.Dense(25, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(30, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

In [ ]:
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(patience = 10, min_delta = 0.001, 
                                               restore_best_weights =True )
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=40,
    epochs=500,
    callbacks=[early_stopping],
    verbose=1, 
)

In [ ]:
predictions =(model.predict(X_test)>0.5).astype("int32")
              
predictions[:10]

94% accuracy

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.evaluate(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
accuracy_score(y_test, predictions)

In [ ]:
print(classification_report(y_test, predictions))